<a href="https://colab.research.google.com/github/adimis-ai/NLP_Projects/blob/main/Multi_Docs_Extractive_Q%26A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-rvwxq9ae/farm-haystack_b7f6a7dabf6a48c49670df0dc4e78db2
  Resolved https://github.com/deepset-ai/haystack.git to commit db6e5754cd71dd822f466bebe3882dc61de00c44
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━

  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-rvwxq9ae/farm-haystack_b7f6a7dabf6a48c49670df0dc4e78db2


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [4]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [5]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [6]:
import time
time.sleep(30)

In [7]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [8]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first fetch some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/tutorial1"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
print(docs[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip to 'data/tutorial1'
INFO:haystack.utils.preprocessing:Converting data/tutorial1/102_The_Princess_and_the_Queen.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/365_A_Song_of_Ice_and_Fire_Roleplaying.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/229_Game_of_Thrones.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/136_Game_of_Thrones__Season_8__soundtrack_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/402_A_Knight_of_the_Seven_Kingdoms__Game_of_Thrones_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/145_Elio_M._García_Jr._and_Linda_Antonsson.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/400_Winterfell__Game_of_Thrones_episode_.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial1/306_The_Dance_of_Dragons.txt
INFO:haystack.utils.pr

[<Document: {'content': '\'\'\'\'\'The Princess and the Queen, or, the Blacks and the Greens\'\'\'\'\' is a novella by George R. R. Martin, published in the 2013 Tor Books anthology \'\'Dangerous Women\'\'. Set in the Westeros of Martin\'s \'\'A Song of Ice and Fire\'\' series, 200 years before the events of \'\'A Game of Thrones\'\' (1996), it chronicles the "continent-burning warfare" (called the "Dance of Dragons") that explodes between Targaryen Princess Rhaenyra and her stepmother, Queen Alicent.\nThe work is presented as the writing of the fictional Archmaester Gyldayn, also the "author" of Martin\'s 2014 novella \'\'The Rogue Prince\'\', a direct prequel to \'\'The Princess and the Queen\'\'.', 'content_type': 'text', 'score': None, 'meta': {'name': '102_The_Princess_and_the_Queen.txt'}, 'embedding': None, 'id': '5417b82a86791a47047d765f53f60dde'}>, <Document: {'content': "\n==Plot==\nWhen King Viserys I Targaryen dies, his widow Queen Alicent has their eldest son Prince Aegon c

In [9]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [10]:
from haystack.nodes import FARMReader

# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [11]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [12]:
from haystack.utils import print_answers

def question(QUERY):
  prediction = pipe.run(
      query=QUERY, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
  )
  print_answers(prediction, details="minimum")

In [13]:
usr_input = question(input("Enter your Question:  "))
print(usr_input)

Enter your Question:  who is arya stark's father


Inferencing Samples: 100%|██████████| 1/1 [00:17<00:00, 17.09s/ Batches]


Query: who is arya stark's father
Answers:
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'King Robert',
        'context': 'ordered by the Lord of Light. Melisandre later reveals to '
                   'Gendry that King Robert was his father and she is bringing '
                   'him to meet his uncle, King Stann'},
    {   'answer': 'Ned Stark',
        'context': 'rl disguised as a boy all along and is surprised to learn '
                   "she is Arya, Ned Stark's daughter. After the Goldcloaks "
                   'get help from Ser Amory Lorch and hi'},
    {   'answer': 'Balon',
        'context': "sgusted, Robb acquiesces to Theon's further captivity, as "
                   "Theon's father Balon has recently died and Theon's absence "
                   'pr